# 19013880 - Finding the ground state energy:

In [2]:
#for interactive plots:
%matplotlib notebook

#import modules:
import numpy as np
import matplotlib.pyplot as plt

We can find the boundary conditions of the infinite square well, analytically. The energy and wavefunction of the electron that can reside in the well is given by the one-dimensional Schrodinger equation [2]:

$$
-\frac{\hbar^2}{2m} \frac{d^2\psi}{dx^2} + V(x)\psi(x) = E\psi(x)
$$

 where $V(x)$ is the potential of the electron, $\psi$ is the wavefunction of the electron, $h$ is the reduced Plank's constant, $m$ is the mass of the electron, $x$ is the 1D position and $E$ is the energy of the electron.
 
First, lets code the constants:

In [3]:
m = 9.109383702*(10**(-31)) #mass of electron in kg
e = 1.602176634*(10**(-19)) #charge of electron in C
h_bar = 1.054571817*(10**(-34)) #reduced Plank's constant in Js
a = 5*(10**(-11)) #one boundary of infinite square well... width = 2a

In an infinite square well, $V(x)$ is defined as:

$$ V_x = \begin{cases} 0 & \text{if } -a \leq x \leq +a, \\ 
\infty  & \text{if } |x| > a, \end{cases} 
$$

However, from $-a$ to $a$ the potential is 0:

In [4]:
def V(x):
    '''
    Potential function
    Inputs: x
    Returns: 0.0 for all inputs x
    
    '''
    return 0.0

Solving the TISE, gives 2 differential equations [2]:

$$
\frac{d \psi}{dx } = \phi, \\
\frac{d \phi}{dx } = \frac{2m}{\hbar^2} \left[V(x) - E \right] \psi(x),
$$

where $\phi$ is the eigenvalue. The differential can be solvedn numerically, using the 4th order Runge-Kutta function, to give $E$ at the boundary $x=a$ Define the two differential equations:

In [5]:
def schrodinger(r, x, E):
    '''Schrodinger differential equations
    Inputs: r (vector of psi and phi), x (1D position), E (energy)
    Outputs: [d_psi,d_phi] (both differential equations in an array)
    '''
    
    #psi and phi will be a vector r:
    psi = r[0] #psi is the first element 
    phi = r[1] #phi is second
    
    #the differential equations:
    d_psi = phi
    d_phi = ((2*m)/h_bar**2)*(V(x)-E)*psi
    
    return np.array([d_psi,d_phi]) #return as an array of the differential equations


We will now define an a array of $x$ values:

In [6]:
xstart = -a #starting boundary       
xend = a    #ending boundary       
N = 1000   #number of x points           
h = (xend - xstart)/N #size of each x point

xpoints = np.arange(xstart, xend, h) #make x points into an array

Using the 4th order Runge-Kutta function, we can numerically solve the differential equations shown above. We will first define an initial $\mathbf r$ and iterate using the Runge-Kutta function and the function that defines the differential equations (`schrodinger`):

In [7]:
#The follwing code is pasted from 10th code cell of 'PHAS0029 Session 8 Finding roots, and boundary value problems', Author: Louise Dash louise.dash@ucl.ac.uk
#[1]

def RungeKutta2d(xpoints, E):
    '''Fourth-order Runge-Kutta rewritten for two simultaneous equations
    Inputs: xpoints: array of x values, E: energy
    Outputs: [psi_points, phi_points]: solutions for psi(x) and phi(x), numpy arrays one longer than xpoints'''
    
    initial_psi = 0.0 
    initial_phi = 1.0
    r = np.array([initial_psi,initial_phi]) #array of initial r

    
    psi_points = [] #initialise empty arrays
    phi_points = []
    
    for x in xpoints: #loops over all xpoints up to (xend-h)
        psi_points.append(r[0])
        phi_points.append(r[1])

        k1 = h*schrodinger(r, x, E) 
        k2 = h*schrodinger(r+0.5*k1, x+0.5*h, E)
        k3 = h*schrodinger(r+0.5*k2, x+0.5*h, E)
        k4 = h*schrodinger(r+k3, x+h, E)
        r = r + (k1 + 2*k2 + 2*k3 + k4)/6
    
    # these next two lines calculate for the point at x = xend!
    psi_points.append(r[0])
    phi_points.append(r[1])

    
    return np.array([psi_points]) #only need psi values


Use the Secant method to find the energy value that leads to $\psi(x) = 0$ at $x=a$. First, guess two values for $E$ and find the corresponding wavefunctions (represented by Runge-Kutta). Then use a while loop to perform the Secant method:


In [8]:
#The follwing code is pasted from 11th code cell of 'PHAS0029 Session 8 Finding roots, and boundary value problems', Author: Louise Dash louise.dash@ucl.ac.uk
[1]

#Initial Conditions:
E1 = 0.0    # guess 1 of initial E1 (J)
E2 = e      # guess 2 of initial E2 (J) 

wf1 = RungeKutta2d(xpoints, E1)[0,N]
wf2 = RungeKutta2d(xpoints, E2)[0,N]

tolerance = e/1000 # set the tolerance for convergence

#Iterate Secant method:
while abs(E2-E1) > tolerance: 
    E3 = E2 - wf2*(E2-E1)/(wf2-wf1)  #Secant method equation
    
    #update initial energies for the next iteration:
    E1 = E2 
    E2 = E3 
    
    # and recalculate wavefunctions:
    wf1 = RungeKutta2d(xpoints, E1)[0,N]
    wf2 = RungeKutta2d(xpoints, E2)[0,N]
    
    print("...difference between E1,E2 is", abs(E2-E1), "\tCondition met?", abs(E2-E1) < tolerance) 

print() #space
print("The energy needs to be", E3/e, "eV") #divide by e to convert from J to eV

...difference between E1,E2 is 3.550738126956973e-18 	Condition met? False
...difference between E1,E2 is 1.2815251256815889e-18 	Condition met? False
...difference between E1,E2 is 7.959729129154386e-19 	Condition met? False
...difference between E1,E2 is 2.0858055385155895e-19 	Condition met? False
...difference between E1,E2 is 2.6836689494863816e-20 	Condition met? False
...difference between E1,E2 is 7.93588897073721e-22 	Condition met? False
...difference between E1,E2 is 2.7330653648344004e-24 	Condition met? True

The energy needs to be 37.60301621189953 eV


To find the analytical solution to the energy at $\psi(x) = 0$ at $x=a$, we use the equation [2]:

$$
E_n = \frac{\pi^2 \hbar^2 n^2}{2m (2a)^2}
$$

In [9]:
n=1 #define n
En = (np.pi**2)*(n**2)*(h_bar**2)/(2*m*((2*a)**2))
print("The analytically solved energy is", En/e, "eV")

The analytically solved energy is 37.60301621352959 eV


## Bibliography

[1] * Dash, L. 2021. _" PHAS0029 Session 8 Finding roots, and boundary value problems"_. \[Online\] UCL Moodle resource. Available from   https://moodle.ucl.ac.uk/course/view.php?id=6766 \[accessed 9th March 2021\]

[2] * Dash, L. 2021. _" Supplementary notebook for the Final Assignment"_. \[Online\] UCL Moodle resource. Available from   https://moodle.ucl.ac.uk/course/view.php?id=6766 \[accessed 9th March 2021\] 
